In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from ResNet import ResNet, ResidualBlock, evaluate, accuracy

In [2]:
ResNet18 = [2, 2, 2, 2]
ResNet50 = [3, 4, 6, 3]
ResNet101 = [3, 4, 23, 3]
ResNet152 = [3, 8, 36, 3]

In [11]:
x = torch.randn(1, 3, 224, 224)
model = ResNet(block=ResidualBlock, layers=ResNet18)

out = model(x)
out.shape

torch.Size([1, 10])

In [12]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor()
])

In [13]:
train_dataset = torchvision.datasets.CIFAR10(root='../datasets/CIFAR10/', train=True, transform=transforms, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='../datasets/CIFAR10/', train=False, transform=transforms, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

In [15]:
def fit(epochs, model, train_loader, val_loader, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(model.parameters(), 6e-5)
    for epoch in range(epochs):
        lrs = []
        loss = 0
        acc = 0
        for batch in tqdm.tqdm(train_loader):
            loss, acc = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print("Epoch [{}], loss: {:.4f}, acc: {:.4f}".format(epoch, loss, acc))
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [16]:
device = 'cuda'

In [17]:
model = model.to(device)

In [18]:
import tqdm
history = fit(5, model, train_dl, test_dl)

100%|██████████| 782/782 [04:57<00:00,  2.63it/s]


Epoch [0], loss: 1.0964, acc: 0.6875
Epoch [0], val_loss: 1.1847, val_acc: 0.5742


100%|██████████| 782/782 [04:59<00:00,  2.61it/s]


Epoch [1], loss: 0.9599, acc: 0.6875
Epoch [1], val_loss: 0.9560, val_acc: 0.6608


100%|██████████| 782/782 [04:59<00:00,  2.61it/s]


Epoch [2], loss: 0.7479, acc: 0.7500
Epoch [2], val_loss: 0.7580, val_acc: 0.7358


100%|██████████| 782/782 [04:59<00:00,  2.62it/s]


Epoch [3], loss: 0.7914, acc: 0.5625
Epoch [3], val_loss: 0.6616, val_acc: 0.7751


100%|██████████| 782/782 [04:59<00:00,  2.61it/s]


Epoch [4], loss: 0.7149, acc: 0.6875
Epoch [4], val_loss: 0.6502, val_acc: 0.7779
